In [17]:
import torch
from torch import nn 
class NeuralNetwork(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, 30),
            nn.ReLU(),

            nn.Linear(30, 20),
            nn.ReLU(),

            # output layer
            nn.Linear(20, num_outputs)            
        )
    def forward(self, x):
        logits = self.layers(x)
        return logits

In [18]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [19]:
# check the number of trainable parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("total number of trainable model parameters:", num_params)

total number of trainable model parameters: 2213


In [20]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [ ]:
# with grad
torch.manual_seed(123)
X = torch.rand((1,50))
out = model(X)
print(out)

tensor([[-0.1928,  0.0651,  0.0459]], grad_fn=<AddmmBackward0>)


In [22]:
# without grad, we have to call softmax function manually
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)


tensor([[0.2806, 0.3632, 0.3562]])
